## Basic Importing


In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Connecting the datasets

In [ ]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/Data/train'
valid_path = '/content/drive/MyDrive/Data/test'
# useful for getting number of classes
folders = glob('/content/drive/MyDrive/Data/train/*')

## Creating the Xception Model

In [ ]:
xception= Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in xception.layers:
  layer.trainable = False

x = Flatten()(xception.output)
#x = Dense(1024, activation='relu')
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=xception.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

## Creating the ImageDataGenerator and the Training and Test Sets

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

## Fitting the model and saving the model

In [ ]:
model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
model.save('model2.h5')

## Importing for testing the model

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models
import numpy as np

## Loading and performing some Image Processing

In [ ]:
model = models.load_model('model2.h5')
test_image=image.load_img('test_pred/pneumonia.jpg',target_size = (224, 224))
test_image=image.img_to_array(test_image) #converts it into 3d array
test_image=np.expand_dims(test_image,axis=0)

## Predicting the result on the entered image

In [ ]:
prediction=model.predict([test_image])
count=0
   
for i in range(len(prediction[0])):
    if(prediction[0][i]==0):
        count+=1
    elif(prediction[0][i]==1):
        break
if(count==0):
    pred="COVID_19"
elif(count==1):
    pred="Normal Patient"
else:
    pred="Pneumonia"    
        
print(prediction[0])        
print(pred)